## Домашнее задание 1

Стребежев Игорь

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
% matplotlib inline
%config InlineBackend.figure_format = 'retina'


df = pd.read_csv('spambase.csv')
df.head()

Y = df.label.values
X = df[df.columns.drop('label')].as_matrix()

### task 1

Реализуйте алгоритм $\texttt{kNN}$ классификации по $k$ ближайшим соседям, используя простое
евклидовое расстояние.

In [3]:
from scipy.spatial.distance import cdist


def knn(X, Y, x: np.ndarray, k=1):
    # compute distances between x and each vector in X
    dists = cdist([x], X, 'euclidean')[0]
    # take the k nearest vectors
    nearest = dists.argpartition(k, axis=0)[:k]
    # vote for the most common class
    return int(Y[nearest].sum() >= nearest.size / 2.0)

### task 2

Вычислите значение метрики $\texttt{LOO}$ для всех вариантов количества соседей $k$ от 1 до 10.

In [4]:
def loo(X, Y):
    N = Y.size

    # for each row R: make a hole at R in the X
    holes = np.arange(1, N) - np.tri(N, N - 1, k=-1, dtype=bool)

    for k in range(1, 11):
        errs = (knn(X[ixs], Y[ixs], X[row], k) != Y[row] for (row, ixs) in enumerate(holes))
        rate = sum(errs) * 100.0 / N
        print(f'k={str(k) + ",":<3} error: {rate:.5f}%')

In [387]:
loo(X, Y)

k=1,  error: 16.95284%
k=2,  error: 20.21300%
k=3,  error: 18.56118%
k=4,  error: 19.38709%
k=5,  error: 18.58292%
k=6,  error: 19.32189%
k=7,  error: 19.56097%
k=8,  error: 19.80004%
k=9,  error: 20.27820%
k=10, error: 20.45208%


### task 3

Реализуйте алгоритм $\texttt{RadiusNeighbours}$ классификации по соседям, лежащим на расстоянии меньшем $r$ (радиус).

In [5]:
def radNN(X, Y, x: np.ndarray, r=1.0):
    # compute distances between x and each vector in X
    dists = cdist([x], X, 'euclidean')[0]
    # take all in the circle of radius r
    nearest = (dists < r).nonzero()[0]
    # vote for the most common class
    return int(Y[nearest].sum() >= nearest.size / 2.0)

### task 4

Найдите лучший радиус с помощью тернарного поиска.

In [6]:
# find minimum of function f on [left, right] segment
def ternarySearch(f, left, right, absolutePrecision=0.1):
    while True:
        #left and right are the current bounds; the minimum is between them
        if abs(right - left) < absolutePrecision:
            return (left + right)/2

        leftThird  = left + (right - left)/3
        rightThird = right - (right - left)/3

        if f(leftThird) > f(rightThird):
            left  = leftThird
        else:
            right = rightThird
            

def findBestRadius(X, Y, absolutePrecision=0.1):
    N = Y.size
    
    # for each row R: make a hole at R in the X
    holes = np.arange(1, N) - np.tri(N, N - 1, k=-1, dtype=bool)
    
    def rloo(rad):
        errs = (radNN(X[ixs], Y[ixs], X[row], rad) != Y[row] for (row, ixs) in enumerate(holes))
        rate = sum(errs) * 100.0 / N
        print(f'r={rad:6.2f}  error: {rate}%')
        return rate
    
    distances = cdist(X, X, 'euclidean')
    best_radius  = ternarySearch(rloo, distances.min(), distances.max(), absolutePrecision)
    print(f'best radius equals {best_radius}, test:')
    rloo(best_radius)

In [425]:
findBestRadius(X, Y)

r=5280.01  error: 39.33927407085416%
r=10560.01  error: 39.36100847641817%
r=3520.00  error: 39.295805259726144%
r=7040.01  error: 39.295805259726144%
r=2346.67  error: 38.49163225385786%
r=4693.34  error: 39.33927407085416%
r=1564.45  error: 37.83960008693762%
r=3128.89  error: 38.882851554009996%
r=1042.96  error: 36.23125407520104%
r=2085.93  error: 38.23081938708976%
r=695.31  error: 34.34036079113236%
r=1390.62  error: 36.68767659204521%
r=463.54  error: 33.81873505759617%
r=927.08  error: 35.18800260812867%
r=309.03  error: 32.579873940447726%
r=618.05  error: 34.27515757444034%
r=206.02  error: 31.840904151271463%
r=412.04  error: 33.36231254075201%
r=137.35  error: 29.906542056074766%
r=274.69  error: 32.18865464029559%
r= 91.56  error: 29.55879156705064%
r=183.13  error: 31.232340795479242%
r= 61.04  error: 27.55922625516192%
r=122.08  error: 29.928276461638774%
r= 40.69  error: 26.885459682677677%
r= 81.39  error: 29.145837861334492%
r= 27.13  error: 26.646381221473593%
r= 54

### task 5

Нормализуйте датасет так, чтобы все признаки лежали в отрезке $[0, 1]$ и повторите вычисления, сделанные в пунктах 2 и 4, на новом датасете.

In [52]:
nXX = (X - X.min(0)) / X.ptp(0)

In [8]:
loo(nXX, Y)

k=1,  error: 8.78070%
k=2,  error: 10.45425%
k=3,  error: 9.49794%
k=4,  error: 10.25864%
k=5,  error: 9.51967%
k=6,  error: 9.86742%
k=7,  error: 9.93262%
k=8,  error: 9.93262%
k=9,  error: 10.19344%
k=10, error: 10.30211%


In [9]:
findBestRadius(nXX, Y, absolutePrecision=0.005)

r=  0.93  error: 39.295805259726144%
r=  1.86  error: 39.404477287546186%
r=  0.62  error: 36.752879808737234%
r=  1.24  error: 39.33927407085416%
r=  0.41  error: 26.47250597696153%
r=  0.83  error: 38.90458595957401%
r=  0.28  error: 19.908715496631167%
r=  0.55  error: 33.86220386872419%
r=  0.18  error: 21.191045424907628%
r=  0.37  error: 23.103673114540317%
r=  0.12  error: 28.47207128885025%
r=  0.25  error: 18.974136057378832%
r=  0.20  error: 19.952184307759183%
r=  0.29  error: 19.647902629863072%
r=  0.26  error: 18.930667246250817%
r=  0.31  error: 20.60421647467942%
r=  0.24  error: 19.19148011301891%
r=  0.28  error: 19.908715496631167%
r=  0.23  error: 19.77830906324712%
r=  0.25  error: 19.148011301890893%
r=  0.24  error: 19.148011301890893%
r=  0.26  error: 19.061073679634863%
r=  0.26  error: 19.25668332971093%
r=  0.27  error: 19.626168224299064%
r=  0.25  error: 19.234948924146924%
r=  0.26  error: 18.930667246250817%
r=  0.26  error: 19.017604868506847%
r=  0.26  

### task 6

Реализуйте алгоритм $\texttt{WkNN}$ с весом $\max\left[0, \frac{r - \rho(x_i, x)}{r}\right]$, и подберите лучшую константу $r$.

In [66]:
def wknn(X, Y, x: np.ndarray, k, r):
    Y = Y * 2 - 1  # change answers to {-1, 1}
    dists = cdist([x], X, 'euclidean')[0]
    nearest = dists.argpartition(k, axis=0)[:k]
    weights = np.maximum(0, (r - dists[nearest]) / r)
    return int((Y[nearest] * weights).sum() >= 0)
    

def findR(X, Y, k=6, absolutePrecision=0.1):
    N = Y.size
    
    # for each row R: make a hole at R in the X
    holes = np.arange(1, N) - np.tri(N, N - 1, k=-1, dtype=bool)
    
    def rloo(r, show=False):
        errs = (wknn(X[ixs], Y[ixs], X[row], k, r) != Y[row] for (row, ixs) in enumerate(holes))
        rate = sum(errs) * 100.0 / N
        if show: print(f'r={r:8.3f}  error: {rate}%')
        return rate
    
    distances = cdist(X, X, 'euclidean')
    best_radius = ternarySearch(rloo, distances.min(), 100 * distances.max(), absolutePrecision)
    print(f'k={k}, best ', end='')
    rloo(best_radius, show=True)

In [67]:
for k in range(1, 11):
    findR(nXX, Y, k)

k=1, best r=   0.902  error: 8.737231036731146%
k=2, best r=   0.902  error: 8.737231036731146%
k=3, best r=   0.903  error: 9.324059986959357%
k=4, best r=   0.934  error: 8.845903064551184%
k=5, best r=   0.934  error: 9.345794392523365%
k=6, best r=   0.902  error: 8.932840686807216%
k=7, best r=   0.933  error: 9.737013692675506%
k=8, best r=   0.902  error: 9.150184742447294%
k=9, best r=   0.615  error: 9.758748098239513%
k=10, best r=   0.934  error: 9.628341664855466%


Заметим, что обычный $\texttt{kNN}$ имел следующие показатели (на 0.5-1.5% хуже):

```
k=1,  error: 8.78070%
k=2,  error: 10.45425%
k=3,  error: 9.49794%
k=4,  error: 10.25864%
k=5,  error: 9.51967%
k=6,  error: 9.86742%
k=7,  error: 9.93262%
k=8,  error: 9.93262%
k=9,  error: 10.19344%
k=10, error: 10.30211%
```

### task 7

Реализуйте алгоритм $\texttt{KDTree}$ и сравните время работы.